# Decision Tree Lab

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [ ]:
import math 
#--------NODE CLASS-------------
class Node():

  def __init__(self, data_array, attr_count=None, parent_entropy=None):
    """
    Args:
        parent_entropy [float]: the entropy of the parent node
        data_array [2D numpy array]: this is an array of the input and target data that only that has been sliced to only include elements filtered by its' parents
        attr_count [int]: this is the value from the count list that corresponds to the attribute being considered (the number of attributes in the category remaining)
    """
    self.parent_entropy = parent_entropy
    self.data_array_of_parent = data_array
    self.attr_count = attr_count 
    self.proportions = []
    self.avrg_entropies_possible_children = []

  def calc_parent_entropy(self):
    parent_class_props = self.proportions[-1] # the targets are on the last column
    self.parent_entropy = self.calc_entropy(parent_class_props)
    return self

  def calc_proportions(self):
    num_cols = self.data_array_of_parent.shape[1]
    for i in range(num_cols):
      class_counts = np.bincount(self.data_array_of_parent[:,i])
      class_props = class_counts / np.sum(class_counts)
      self.proportions.append(class_props)
    return self

  def calc_entropy(self,proportions_tup):
    num_elements = len(proportions_tup)
    entropy = 0
    for i in range(num_elements):
      entropy += -proportions_tup[i]*math.log2(proportions_tup[i])
    return entropy

  def calc_average_entropies(self):
    for child in range(len(self.proportions)):
      child_props = self.proportions[child]
      entropy = 0
      for attr in range(child_props.shape[0]):
        data_array = self.data_array_of_parent[self.data_array_of_parent[:,child] == attr]
        attr_props = np.bincount(data_array[:])
        entropy += child_props[attr]

  def get_parent_entropy(self):
    return self.parent_entropy

  def get_proportions(self):
    return self.proportions
  

In [ ]:
HW_data = np.array([['Y', 'Thin', 'N', 'Great'],
                    ['N','Deep', 'N', 'Bad'],
                    ['N', 'Stuffed', 'Y','Good'],
                    ['Y', 'Stuffed','Y','Great'],
                    ['Y','Deep','N','Good'],
                    ['Y','Deep','Y','Great'],
                    ['N','Thin','Y','Good'],
                    ['Y','Deep','N','Good'],
                    ['N','Thin','N','Bad']])
# N = 0; Y = 1; 
# Deep = 0; Stuffed = 1; Thin = 2
# Bad = 0; Good = 1; Great = 2

HW_data[HW_data == 'Y'] = 1
HW_data[HW_data == 'N'] = 0
HW_data[HW_data == 'Deep'] = 0
HW_data[HW_data == 'Stuffed'] = 1
HW_data[HW_data == 'Thin'] = 2
HW_data[HW_data == 'Bad'] = 0
HW_data[HW_data == 'Good'] = 1
HW_data[HW_data == 'Great'] = 2
HW_data = HW_data.astype(int)
print(HW_data)

node = Node(HW_data)
node.calc_proportions().calc_parent_entropy()
print('parent entropy: ', node.get_parent_entropy())
print('proportions: ', node.get_proportions())

[[1 2 0 2]
 [0 0 0 0]
 [0 1 1 1]
 [1 1 1 2]
 [1 0 0 1]
 [1 0 1 2]
 [0 2 1 1]
 [1 0 0 1]
 [0 2 0 0]]
parent entropy:  1.5304930567574826
proportions:  [array([0.44444444, 0.55555556]), array([0.44444444, 0.22222222, 0.33333333]), array([0.55555556, 0.44444444]), array([0.22222222, 0.44444444, 0.33333333])]


In [ ]:
def calc_entropy(proportions_tup):
  num_elements = len(proportions_tup)
  entropy = 0
  for i in range(num_elements):
    entropy += -proportions_tup[i]*math.log2(proportions_tup[i])
  return entropy

def calc_avrg_entropy(proportions_list):
  entropies = calc_entropy(proportions_list)
  

In [ ]:
HW_data = np.array([['Y', 'Thin', 'N', 'Great'],
                    ['N','Deep', 'N', 'Bad'],
                    ['N', 'Stuffed', 'Y','Good'],
                    ['Y', 'Stuffed','Y','Great'],
                    ['Y','Deep','N','Good'],
                    ['Y','Deep','Y','Great'],
                    ['N','Thin','Y','Good'],
                    ['Y','Deep','N','Good'],
                    ['N','Thin','N','Bad']])
# N = 0; Y = 1; 
# Deep = 0; Stuffed = 1; Thin = 2
# Bad = 0; Good = 1; Great = 2

HW_data[HW_data == 'Y'] = 1
HW_data[HW_data == 'N'] = 0
HW_data[HW_data == 'Deep'] = 0
HW_data[HW_data == 'Stuffed'] = 1
HW_data[HW_data == 'Thin'] = 2
HW_data[HW_data == 'Bad'] = 0
HW_data[HW_data == 'Good'] = 1
HW_data[HW_data == 'Great'] = 2
HW_data = HW_data.astype(int)
print(HW_data)

counts = [2,3,2,3]
proportions_list = []
for col in range(len(counts)):
  prop = np.bincount(HW_data[:,col])
  prop = prop / np.sum(prop)
  proportions_list.append(prop)
print(proportions_list)

# children_entropies = []
# for i in range(len(proportions_list)):
#   children_entropies.append(calc_entropy(proportions_list[i]))
# print('children_entropies', children_entropies)

zero_meat = HW_data[HW_data[:,0] == 0] 
print(zero_meat)
prop = np.bincount(zero_meat[:,-1])
print(prop)

# print(HW_data)
# counts = [2,3,2,3]
# el_counts_col = []
# for col in range(len(counts)):
#   elem_counts = ()
#   total_rows = HW_data.shape[0]
#   for j in range(counts[col]):
#     attr_count = np.sum(HW_data[:,col] == j)
#     elem_counts = elem_counts + (attr_count/total_rows,)
#   el_counts_col.append(elem_counts)
# print(el_counts_col)

print('entropy of root node',calc_entropy(proportions_list[-1]))



[[1 2 0 2]
 [0 0 0 0]
 [0 1 1 1]
 [1 1 1 2]
 [1 0 0 1]
 [1 0 1 2]
 [0 2 1 1]
 [1 0 0 1]
 [0 2 0 0]]
[array([0.44444444, 0.55555556]), array([0.44444444, 0.22222222, 0.33333333]), array([0.55555556, 0.44444444]), array([0.22222222, 0.44444444, 0.33333333])]
[[0 0 0 0]
 [0 1 1 1]
 [0 2 1 1]
 [0 2 0 0]]
[2 2]
entropy of root node 1.5304930567574826


In [ ]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        

    def fit(self, X, y):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        # 
        return self

    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        pass


    def score(self, X, y):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        return 0



## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset (lenses.arff)](https://byu.instructure.com/courses/14142/files?preview=4622251)

Test your model on the lenses test set: [Debug Test Dataset (lenses_test.arff)](https://byu.instructure.com/courses/14142/files?preview=4622254)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions (pred_lenses.csv)](https://byu.instructure.com/courses/14142/files?preview=4622260)

*NOTE: The [Lenses Prediction (pred_lenses.csv)](https://byu.instructure.com/courses/14142/files?preview=4622260) uses the following encoding: soft=2, hard=0, none=1. If your encoding is different, then your output will be different, but not necessarily incorrect.*

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
tear_prod_rate = normal:
    astigmatism = no:
        age = pre_presbyopic:
            prediction: soft
        age = presbyopic:
            spectacle_prescrip = hypermetrope:
                prediction: soft
            spectacle_prescrip = myope:
                prediction: none
        age = young:
            prediction: soft
    astigmatism = yes:
        spectacle_prescrip = hypermetrope:
            age = pre_presbyopic:
                prediction: none
            age = presbyopic:
                prediction: none
            age = young:
                prediction: hard
        spectacle_prescrip = myope:
            prediction: hard
tear_prod_rate = reduced:
    prediction: none
</pre>

In [ ]:
# Load debug training data 


# Train Decision Tree


# Load debug test data


# Predict and compute model accuracy


# Print the information gain of every split you make.


In [ ]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset (zoo.arff)](https://byu.instructure.com/courses/14142/files?preview=4622270)

Test your model on this dataset: [Evaluation Test Dataset (zoo_test.arff)](https://byu.instructure.com/courses/14142/files?preview=4622274)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [ ]:
# Load evaluation training data


# Train Decision Tree


# Load evaluation test data


# Print out the information gain for every split you make



## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criterion, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [ ]:
# Write a function that implements 10-fold cross validation

##  2.2 Cars Dataset
- Use this [Cars Dataset (cars.arff)](hhttps://byu.instructure.com/courses/14142/files?preview=4622293)
- Make a table for your k-fold cross validation accuracies

*If you are having trouble using scipy's loadarff function (scipy.io.arff.loadarff), try:*

*pip install arff &nbsp;&nbsp;&nbsp;&nbsp;          # Install arff library*

*import arff as arf*                   

*cars = list(arf.load('cars.arff'))   &nbsp;&nbsp;&nbsp;&nbsp;# Load your downloaded dataset (!curl, etc.)*

*df = pd.DataFrame(cars)*  

*There may be additional cleaning needed*

In [ ]:
# Use 10-fold CV on Cars Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values (voting_with_missing.arff)](https://byu.instructure.com/courses/14142/files?preview=4622298)
- Note that you will need to support unknown attributes in the voting data set. 

In [ ]:
# Used 10-fold CV on Voting Dataset

# Report Training and Test Classification Accuracies

# Report Average Test Accuracy

## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

Discuss your results

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e., look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

Discussion Here

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

Discussion Here

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

Discuss how you handled unknown attributes

## 4.1 (10%) Use Scikit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform best on the test set. 

### 4.1.1 sklearn on Voting Dataset
- Use this [Voting Dataset with missing values (voting_with_missing.arff)](https://byu.instructure.com/courses/14142/files?preview=4622298)

In [ ]:
# Use sklearn's Decision Tree to learn the voting dataset

# Explore different parameters

# Report results

Discuss results & compare to your method's results

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the sklearn decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [ ]:
# Use sklearn's Decision Tree on a new dataset

# Experiment with different hyper-parameters

## 5. (5%) Visualize sklearn's decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first few levels (e.g., top 5).

In [ ]:
# Include decision tree visualization here

# Discuss what the model has learned

## 6. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).